#### 1. 什么时候观察负载    
每次发现系统变慢时，我们通常做的第一件事，就是执行 top 或者 uptime 命令，来了解系统的负载情况。  
```shell
$ uptime
 15:49:51 up 648 days,  5:25, 54 users,  load average: 1.31, 2.38, 2.17
```
我相信你对前面的几列比较熟悉，它们分别是当前时间、系统运行时间以及正在登录用户数。而最后三个数字呢，依次则是过去 1 分钟、5 分钟、15 分钟的平均负载（Load Average）。

#### 2. 什么是平均负载   
  平均负载指单位时间内两类进程的数量:   
  * 正在使用 CPU 的进程  
  * 等待 CPU 和等待 I/O的进程     
    等待 CPU 和等待 I/O的进程又叫不可中断状进程. 比如，当一个进程向磁盘读写数据时，为了保证数据的一致性，在得到磁盘回复前，它是不能被其他进程或者中断打断的，这个时候的进程就处于不可中断状态。如果此时的进程被打断了，就容易出现磁盘数据与进程数据不一致的问题。所以，不可中断状态实际上是系统对进程和硬件设备的一种保护机制。 

#### 3. 当平均负载为 2 时，意味着什么呢？
* 在只有 2 个 CPU 的系统上，意味着所有的 CPU 都刚好被完全占用。
* 在 4 个 CPU 的系统上，意味着 CPU 有 50% 的空闲。
* 而在只有 1 个 CPU 的系统中，则意味着有一半的进程竞争不到 CPU。

#### 4. 平均负载为多少时合理
* 首先你要知道系统有几个 CPU   
  可以通过 top 命令或者从文件 /proc/cpuinfo 中读取  
  ```shell
  $ grep 'model name' /proc/cpuinfo | wc -l
  16
  ```
* 然后, 再看3个时间段的平均负载  
  * 如果 1 分钟、5 分钟、15 分钟的三个值基本相同，或者相差不大，那就说明系统负载很平稳。
  * 但如果 1 分钟的值远小于 15 分钟的值，就说明系统最近 1 分钟的负载在减少，而过去 15 分钟内却有很大的负载。
  * 反过来，如果 1 分钟的值远大于 15 分钟的值，就说明最近 1 分钟的负载在增加，这种增加有可能只是临时性的，也有可能还会持续增加下去，所以就需要持续观察。一旦 1 分钟的平均负载接近或超过了 CPU 的个数，就意味着系统正在发生过载的问题，这时就得分析调查是哪里导致的问题，并要想办法优化了。    
  
  
* 在我看来，当平均负载高于 CPU 数量 70% 的时候，你就应该分析排查负载高的问题了。  
  一旦负载过高，就可能导致进程响应变慢，进而影响服务的正常功能。

#### 5. 平均负载案例分析
1. 用到的工具  
  `sysstat`: 监控和分析系统的性能
  `pidstat`: 实时查看进程的 CPU、内存、I/O 以及上下文切换等性能指标
  `mpstat` : 多核 CPU 性能分析工具，实时查看每个 CPU 的性能指标
  `iostat`